In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
path = '../Data/RAWDATA_Covid_Admissions.xlsx'
df = pd.read_excel(path, converters={
    'Date': pd.to_datetime,
    'Admissions': np.float64
})

Calculate net patient intake and no. discharges from Total bed occupancy and Admissions data:

In [ ]:
df['Net_Intake'] = df['Total bed occupancy'].diff()
df['Discharges'] = df['Admissions'] - df['Net_Intake']

In [ ]:
df.head()

In [ ]:
df.describe()

Function to calculate rolling mean given number of periods:

In [ ]:
def rolling_mean(s, n):
    '''
    returns rolling mean of series.
    - s: series
    - n: window length
    '''
    return s.rolling(n).mean()

**Calculate whether change in rolling mean from previous period is statistically significant:**

Use t-test (independent) to check probability that this period and previous period are from different populations (i.e. the mean is statistically significantly different)

In [ ]:
def split_ttest_p(s):
    '''
    returns t-test probability of two halves of a series.
    - s: series
    '''
    s1 = s[0:len(s)//2]
    s2 = s[len(s)//2:]
    return st.ttest_ind(s1, s2, equal_var=False).pvalue

def rolling_ttest(s, n, c=False):
    '''
    returns rolling split t-test result of a provided series.
    - s: series
    - n: window length
    - c: centering
    '''
    if c:
        return s.shift(periods=-(n//2 + 1)).rolling(2*n).apply(split_ttest_p, raw=False)
    else:
        return s.rolling(2*n).apply(split_ttest_p, raw=False)        

Compare Admissions vs Discharges vs. Net Intake --> see **crossover point (when peak occupancy has been reached)**

Sanity check--Check that cumsum(Net_Intake)==Total bed occupancy (this should be true for everything to add up)

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

df1 = df[['Date', 'Admissions', 'Discharges', 'Net_Intake']].copy()

df1.rolling(7, on='Date').mean().plot(
    x='Date', 
    y=['Admissions', 'Discharges', 'Net_Intake'], 
    ax=ax,
    color=['#3e003e50', '#003e3e50', '#006A90'],
    
)

df.plot(x='Date', y='Total bed occupancy', c='#B3193990', ax=ax, secondary_y=True)
ax.right_ax.plot(df1['Date'], df1['Net_Intake'].cumsum(), ls='--', label='cumsum(Net_Intake)', c='#B3193990')
# ax.right_ax.axhline(84, lw=0.75, c='k')

ax.axhline(0, lw=0.25, c='k')
ax.right_ax.axhline(0, lw=0.25, c='k')

ax.set_title(f'CUH Covid: First wave ({7} day rolling means)')
ax.set_xlabel('Date')
ax.set_ylabel('Daily patients')
ax.right_ax.set_ylabel('Bed occupancy')

ax.grid(which='minor', axis='x')
ax.legend(loc='upper left')
ax.right_ax.legend(loc='upper right')

plt.show()

**Observations:**

- Need an admissions buffer to absorb volatility
- **Net intake** is a metric that matters, not just **admissions** and **total bed occupancy**


**Conclusions:**

- Meta level trigger

**Work on making plots interactive:**

- Change number of days for rolling mean
- Include ward opening triggers
- What about amber wards?

In [ ]:
import plotly.graph_objects as go
from ipywidgets import widgets

## Plotly with dynamic traces

In [ ]:
no_days = widgets.IntSlider(
    value=7,
    min=1,
    max=10,
    step=1,
    description='No days:',
    continuous_update=True,
)

centered = widgets.Checkbox(
    value=False,
    description='Centred rolling mean',
    disabled=False
)

p_sig = widgets.FloatSlider(
    value=0.05,
    min=0.01,
    max=0.10,
    step=0.01,
    description='P (sig.):',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)


layout = go.Layout(
    title='CUH Covid Admissions: First wave',
    xaxis={
        'title': 'Date',
        'tickformat': '%d %b',
        'tickmode': 'linear',
        'nticks': 20,
        'tick0': '2020-03-01',
        'dtick': np.timedelta64(7,'D').astype('timedelta64[ms]').astype(int)
    },
    yaxis={
        'title': 'No. Admissions'
    }
)

data = []
data.append(
    go.Scatter(
        name = 'Admissions',
        x = df['Date'],
        y = df['Admissions'],
        mode = 'lines+markers',
        opacity = 0.5
    )
)
data.append(
    go.Scatter(
        name = f'Admissions ({no_days.value} day rolling mean)',
        x = df['Date'],
        y = df['Admissions'].rolling(no_days.value).mean()
    )
)

Admissions_rolling_pttest = rolling_ttest(df['Admissions'], no_days.value, c=centered.value)
mask_sig = Admissions_rolling_pttest < p_sig.value

data.append(
    go.Scatter(
        name = f'p(ttest) < {p_sig.value}',
        x = df['Date'][mask_sig],
        y = df['Admissions'].rolling(no_days.value).mean()[mask_sig],
        mode = 'markers',
        marker={
            'color': 'black'
        }
    )
)

g = go.FigureWidget(data=data, layout=layout)


def response(change):
    y = df['Admissions'].rolling(no_days.value, center=centered.value).mean()
    
    Admissions_rolling_pttest = rolling_ttest(df['Admissions'], no_days.value, c=centered.value)
    mask_sig = Admissions_rolling_pttest < p_sig.value


    with g.batch_update():
        g.data[1].y = y
        g.data[1].name = f'Admissions ({no_days.value} day rolling mean)'
        
        g.data[2].x = df['Date'][mask_sig]
        g.data[2].y = y[mask_sig]
        g.data[2].name = f'p(ttest) < {p_sig.value}'
        

no_days.observe(response, names='value')
centered.observe(response, names='value')
p_sig.observe(response, names='value')

container = widgets.HBox([no_days, p_sig, centered])

widgets.VBox([
    container,
    g
])

In [ ]:
from cuhvid.triggers import get_rolling_mean
columns = ['Admissions']
get_rolling_mean(df, columns, no_days=7)

In [ ]:
no_days = widgets.IntSlider(
    value=7,
    min=1,
    max=10,
    step=1,
    description='No days:',
    continuous_update=True,
)

centered = widgets.Checkbox(
    value=False,
    description='Centred rolling window',
    disabled=False
)

p_sig = widgets.FloatSlider(
    value=0.05,
    min=0.01,
    max=0.10,
    step=0.01,
    description='P (sig.):',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)


layout = go.Layout(
    title='CUH Covid Admissions: First wave',
    xaxis={
        'title': 'Date',
        'tickformat': '%d %b',
        'tickmode': 'linear',
        'nticks': 20,
        'tick0': '2020-03-01',
        'dtick': np.timedelta64(7,'D').astype('timedelta64[ms]').astype(int)
    },
    yaxis={
        'title': 'No. Admissions'
    }
)

data = []
data.append(
    go.Scatter(
        name = 'Admissions delta',
        x = df['Date'],
        y = df['Admissions'].diff(),
        mode = 'lines+markers',
        opacity = 0.5
    )
)
data.append(
    go.Scatter(
        name = f'Admissions std dev. ({no_days.value} day rolling)',
        x = df['Date'],
        y = df['Admissions'].rolling(no_days.value).std()
    )
)

Admissions_rolling_pttest = rolling_ttest(df['Admissions'], no_days.value, c=centered.value)
mask_sig = Admissions_rolling_pttest < p_sig.value

data.append(
    go.Scatter(
        name = f'p(ttest) < {p_sig.value}',
        x = df['Date'][mask_sig],
        y = df['Admissions'].rolling(no_days.value).std()[mask_sig],
        mode = 'markers',
        marker={
            'color': 'black'
        }
    )
)

g = go.FigureWidget(data=data, layout=layout)


def response(change):
    y = df['Admissions'].rolling(no_days.value, center=centered.value).std()
    
    Admissions_rolling_pttest = rolling_ttest(df['Admissions'], no_days.value, c=centered.value)
    mask_sig = Admissions_rolling_pttest < p_sig.value


    with g.batch_update():
        g.data[1].y = y
        g.data[1].name = f'Admissions std dev. ({no_days.value} day rolling)'
        
        g.data[2].x = df['Date'][mask_sig]
        g.data[2].y = y[mask_sig]
        g.data[2].name = f'p(ttest) < {p_sig.value}'
        

no_days.observe(response, names='value')
centered.observe(response, names='value')
p_sig.observe(response, names='value')

container = widgets.HBox([no_days, p_sig, centered])

widgets.VBox([
    container,
    g
])

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(
    df,
    x='Date',
    y=df['Admissions'].rolling(3, center=True).mean(),
    error_y=df['Admissions'].rolling(3, center=True).std(),
    marginal_y='histogram',
    title='CUH Covid admissions (& std dev.) vs. time'
)
fig.show()

In [ ]:
window = 7
fig = px.scatter(
    df,
    x=df['Admissions'].rolling(window, center=True).mean(), # .values[0::window]
    y=df['Admissions'].rolling(window, center=True).std(), # .values[0::window]
    title=f'CUH Covid Admissions vs. Std dev. ({window} day rolling window)',
    labels=dict(x=f'Mean Admissions (for {window} day window)', y=f'Std dev. (for {window} day window)'),
    trendline='ols'
)
fig.show()
# output for 7 days:       std dev = 0.30*mean adm. + 0.60

In [ ]:
df['Admissions'].describe()

In [ ]:
import scipy.stats as st
print(df.Admissions[:-1].shape, df.Admissions[1:].shape)
print(st.pearsonr(df.Admissions[:-1], df.Admissions[1:]))

fig = px.line(
    df,
    x=df.Date[:-1],
    y=[df.Admissions[:-1], df.Admissions[1:]]
)

fig.show()

In [ ]:
df.head()

In [ ]:
df = df.set_index('Date')
df.index = df.index.to_period()

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots()
df.Admissions[:-1].plot(ax=ax)
df.Admissions.shift(-1)[:-1].plot(ax=ax, label='Admissions shifted')
ax.legend()
plt.show()


print(st.pearsonr(df.Admissions[:-1], df.Admissions[1:]))
print(st.pearsonr(df.Admissions[:-1], df.Admissions.shift(-1)[:-1]))

In [ ]:
pd.plotting.autocorrelation_plot(df.Admissions, marker='o', linestyle='None')
ax = plt.gca()

x, y = ax.lines[-1].get_data()
plt.stem(x, y, use_line_collection=True, basefmt='k', linefmt='k')
ax.set_title('Autocorrelation: Admissions (pandas)')
ax.set_xlabel('No. lags')
ax.set_xlim([-0.5, 20.5])
ax.set_xticks(np.arange(0,21,5))
ax.set_yticks(np.arange(-1,1.1,0.2))
ax.grid(None)
ax.set_ylabel('Correlation')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(df.Admissions, alpha=0.05)
ax = plt.gca()
ax.set_title('Autocorrelation: Admissions (statsmodels)')
ax.set_xlabel('No. lags')
ax.set_ylabel('Correlation')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(df.Admissions, alpha=0.05)
ax = plt.gca()
ax.set_title('Partial Autocorrelation: Admissions')
ax.set_xlabel('No. lags')
ax.set_ylabel('Correlation')
plt.show()

In [ ]:
fig, ax = plt.subplots()
df.Admissions.plot(ax=ax)
ax.legend()
plt.show()

In [ ]:
fit_a, fit_loc, fit_scale = st.gamma.fit(df.Admissions.values)
print(fit_a, fit_loc, fit_scale)

a = 1.99

mean, var, skew, kurt = st.gamma.stats(a, moments='mvsk')
print(mean, var, skew, kurt)

x = np.linspace(st.gamma.ppf(0.01, a),
                st.gamma.ppf(0.99, a), 100)

fig, ax = plt.subplots()
ax.plot(x, st.gamma.pdf(x, a), label='gamma pdf')
ax.legend()
plt.show()


In [ ]:
# Widgets
alpha = widgets.FloatSlider(
    value=6.57,
    min=0.1,
    max=10,
    step=0.05,
    description='alpha:',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

loc = widgets.FloatSlider(
    value=0,
    min=-10,
    max=10,
    step=0.1,
    description='loc:',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

scale = widgets.FloatSlider(
    value=6.56,
    min=0.1,
    max=10,
    step=0.01,
    description='scale:',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

peak_adm = widgets.IntSlider(
    value=12,
    min=1,
    max=50,
    step=1,
    description='Peak adm.:',
    continuous_update=True,
)

variability = widgets.IntSlider(
    value=3,
    min=0,
    max=10,
    step=1,
    description='Variability:',
    continuous_update=True,
)

# Layout
layout = go.Layout(
    title='Gamma distribution',
    xaxis={
        'title': 'Day'
    },
    yaxis={
        'title': 'No. admissions'
    }
)

# Data & functions
def get_series():
    x = np.arange(0, 100, 1)
    y = st.gamma.pdf(x, alpha.value, loc=loc.value, scale=scale.value)
    y = y*peak_adm.value/np.max(y)
    
    y_g = np.clip(y + st.uniform.rvs(size=len(x), 
                                     scale=variability.value, 
                                     loc=-variability.value/2), 
                  a_min=0, a_max=None)
    
    return x, y, y_g

x, y, y_g = get_series()

data = [
    go.Scatter(
        name = 'Probability density function',
        x = x,
        y = y,
        mode='lines'
    )
]
data.append(go.Scatter(
    name = 'Generated series',
    x = x,
    y = y_g,
    mode='lines'
))
data.append(go.Scatter(
    name = 'Admissions (first wave)',
    x = x,
    y = df.Admissions.values,
    mode='lines'
))

# Responsive & plot
g = go.FigureWidget(data=data, layout=layout)

def response(change):
    x, y, y_g = get_series()

    with g.batch_update():
        g.data[0].x = x
        g.data[0].y = y
        g.data[1].x = x
        g.data[1].y = y_g
        

alpha.observe(response, names='value')
loc.observe(response, names='value')
scale.observe(response, names='value')
peak_adm.observe(response, names='value')
variability.observe(response, names='value')

widgets.VBox([
    alpha,
    loc,
    scale,
    peak_adm,
    variability,
    g
])

In [ ]:
# Widgets
reset = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset sliders to default values',
    icon='refresh' # (FontAwesome names without the `fa-` prefix)
)
alpha = widgets.FloatSlider(
    value=6.57,
    min=0.1,
    max=10,
    step=0.05,
    description='alpha:',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

loc = widgets.FloatSlider(
    value=0,
    min=-10,
    max=10,
    step=0.1,
    description='loc:',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

scale = widgets.FloatSlider(
    value=6.56,
    min=0.1,
    max=10,
    step=0.01,
    description='scale:',
    disabled=False,
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

peak_adm = widgets.IntSlider(
    value=12,
    min=1,
    max=50,
    step=1,
    description='Peak adm.:',
    continuous_update=True,
)

stddev_coeff = widgets.FloatSlider(
    value=0.3,
    min=0,
    max=1,
    step=0.01,
    description='σ coeff.:',
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

stddev_const = widgets.FloatSlider(
    value=0.6,
    min=0,
    max=1,
    step=0.1,
    description='σ const.:',
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

# Layout
layout = go.Layout(
    title='Covid admissions',
    xaxis={
        'title': 'Day'
    },
    yaxis={
        'title': 'No. admissions'
    }
)

layout2 = go.Layout(
    title='Cumulative Covid admissions',
    xaxis={
        'title': 'Day'
    },
    yaxis={
        'title': 'Cumulative no. admissions'
    }
)

# Data & functions
std_dev = lambda mean_adm : stddev_coeff.value*mean_adm + stddev_const.value

def get_series():
    x = np.arange(0, 92, 1)
    y = st.gamma.pdf(x, alpha.value, loc=loc.value, scale=scale.value)
    y = y*peak_adm.value/np.max(y)
    
#     y_noise = 1.96 * std_dev(y) * st.uniform.rvs(size=len(y), scale=1, loc=-0.5)
    y_noise = std_dev(y) * st.norm.rvs(size=len(y))
    
    y_g = np.clip(y + y_noise, a_min=0, a_max=None)
    
    y_l = np.clip(y - 1.96*std_dev(y), a_min=0, a_max=None)
    y_u = np.clip(y + 1.96*std_dev(y), a_min=0, a_max=None)
    
    return x, y, y_g, y_l, y_u

x, y, y_g, y_l, y_u = get_series()

data = []
data.append(go.Scatter(
    name = 'Probability density function',
    x = x,
    y = y_l,
    mode='lines',
    line=dict(width=0),
    showlegend=False
))
data.append(go.Scatter(
    name = 'Probability density function',
    x = x,
    y = y,
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
    fill='tonexty',
    fillcolor='rgba(0, 176, 246, 0.3)',
))
data.append(go.Scatter(
    name = 'Probability density function',
    x = x,
    y = y_u,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(0, 176, 246, 0.3)',
    showlegend=False
))
data.append(go.Scatter(
    name = 'Generated series',
    x = x,
    y = y_g,
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data.append(go.Scatter(
    name = 'Admissions (first wave)',
    x = x,
    y = df.Admissions.values,
    mode='lines',
    line=dict(color='rgb(98, 76, 171)'),
))

data2 = []
data2.append(go.Scatter(
    name = 'Probability density function',
    x = x,
    y = y.cumsum(),
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
))
data2.append(go.Scatter(
    name = 'Generated series',
    x = x,
    y = y_g.cumsum(),
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data2.append(go.Scatter(
    name = 'Admissions (first wave)',
    x = x,
    y = df.Admissions.cumsum().values,
    mode='lines',
    line=dict(color='rgb(98, 76, 171)'),
))

# Responsive & plot
g = go.FigureWidget(data=data, layout=layout)
g2 = go.FigureWidget(data=data2, layout=layout2)

def response(change):
    x, y, y_g, y_l, y_u = get_series()

    with g.batch_update():
        g.data[0].y = y_l
        g.data[1].y = y
        g.data[2].y = y_u
        g.data[3].y = y_g
    with g2.batch_update():
        g2.data[0].y = y.cumsum()
        g2.data[1].y = y_g.cumsum()
    
    return
        
def reset_values(change):
    alpha.value = 6.57
    loc.value = 0
    scale.value = 6.56 
    peak_adm.value = 12
    stddev_coeff.value = 0.3
    stddev_const.value = 0.6
    response(None)
    return

reset.on_click(reset_values)
alpha.observe(response, names='value')
loc.observe(response, names='value')
scale.observe(response, names='value')
peak_adm.observe(response, names='value')
stddev_coeff.observe(response, names='value')
stddev_const.observe(response, names='value')

gamma_control = widgets.HBox([
    alpha,
    loc,
    scale,
])
magnitude_control = widgets.HBox([
    peak_adm,
    stddev_coeff,
    stddev_const,
])
widgets.VBox([
    reset,
    gamma_control,
    magnitude_control,
    g,
    g2
])

# Ward scenarios

In [ ]:
path = '../Data/RAWDATA_Ward_Scenarios.xlsx'
df_wards = pd.read_excel(path)

In [ ]:
df_wards.head()

In [ ]:
df_wards['AB_change'] = df_wards.A_color != df_wards.B_color
df_wards['BC_change'] = df_wards.B_color != df_wards.C_color
df_wards.head()

In [ ]:
df_wards['AB_change_no'] = None

change_no = np.arange(len(df_wards[df_wards.AB_change]))
np.random.shuffle(change_no)
df_wards.loc[df_wards.AB_change, 'AB_change_no'] = change_no

df_wards[df_wards.AB_change]

In [ ]:
df_wards['BC_change_no'] = None

change_no = np.arange(len(df_wards[df_wards.BC_change]))
np.random.shuffle(change_no)
df_wards.loc[df_wards.BC_change, 'BC_change_no'] = change_no

df_wards[df_wards.BC_change]

In [ ]:
df_wards['state_color'] = df_wards.A_color
df_wards['state_no_beds'] = df_wards.A_no_beds

In [ ]:
df_wards